In [25]:

%pip -q install openai


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/

In [105]:
# --- Colab setup: installs ---
!pip -q install fastapi uvicorn nest-asyncio pyngrok==7.* websockets==12.* requests pydub numpy openai==1.* python-dotenv ffmpeg-python

# --- Imports ---
import os, json, base64, time, asyncio, nest_asyncio, requests, subprocess, tempfile
from datetime import datetime
from typing import Optional
from collections import deque
from io import BytesIO
import re

from fastapi import FastAPI, Request
from fastapi.responses import PlainTextResponse
import uvicorn
from pyngrok import ngrok
from pydub import AudioSegment
from openai import OpenAI

# --- 🔧 Configuration (EDIT THESE) ---
RECALL_API_KEY    = os.getenv("RECALLAI_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")
NGROK_AUTHTOKEN   = os.getenv("NGROK_AUTHTOKEN")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

MEETING_URL       = os.getenv("MEETING_URL", "https://meet.google.com/qey-yfwq-xeu")  # ← put your Meet link here
WAKE_WORD         = os.getenv("WAKE_WORD", "nemo").lower()                       # ← say this to trigger
OPENAI_MODEL      = os.getenv("OPENAI_MODEL", "gpt-5-nano")                               # any Responses-capable model
OPENAI_TTS_MODEL  = os.getenv("OPENAI_TTS_MODEL", "gpt-4o-mini-tts")
OPENAI_TTS_VOICE  = os.getenv("OPENAI_TTS_VOICE", "alloy")                             # pick a voice you like

# --- States ---
PARTIAL_WINDOW_SEC = 20.0
SILENCE_TIMEOUT_SEC = 2.0  # Time to wait after last speech before finalizing (reduced for faster response)
MAX_QUESTION_CHARS = 400

class CaptureState:
    IDLE = "idle"
    CAPTURING = "capturing"

# State tracking
state = CaptureState.IDLE
captured_utterances = []  # Store only FINAL utterances, not partials
last_utterance_ts: float = 0.0
last_capture_end_ts: float = 0.0

# --- FastAPI app ---
app = FastAPI()

@app.get("/", response_class=PlainTextResponse)
def root():
    return "Recall live bot is up"

# --- Transcript cleaning ---
import re
def clean_transcript_text(raw: str) -> str:
    text = raw.replace("’", "'").replace("`", "'")
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text)
    text = re.sub(r"([.?!])([A-Za-z])", r"\1 \2", text)
    text = re.sub(r"([a-z])([0-9A-Z])", r"\1 \2", text)
    text = re.sub(r"([0-9])([A-Za-z])", r"\1 \2", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# --- Webhook handler ---
@app.post("/rt")
async def rt_webhook(req: Request):
    """
    Handles live transcription events from Recall/Deepgram.
    State machine: IDLE -> hear wake word -> CAPTURING -> collect utterances -> timeout -> process
    Only uses final utterances (transcript.data), ignoring partials to avoid duplicates.
    """
    try:
        data = await req.json()
        event = data.get("event", "")
        if event not in ("transcript.partial_data", "transcript.data"):
            return {"ok": True}

        payload = data.get("data", {}).get("data", {})
        participant = payload.get("participant", {})
        raw_text = payload.get("text") or " ".join(
            w.get("text", "") for w in payload.get("words", [])
        )
        if not raw_text:
            return {"ok": True}

        text = clean_transcript_text(raw_text)
        speaker = participant.get("name", "unknown")
        now = time.time()

        global state, captured_utterances, last_utterance_ts, last_capture_end_ts

        # --- Handle partial data (just for monitoring, don't process) ---
        if event == "transcript.partial_data":
            print(f"[Partial] {speaker}: {text}")
            return {"ok": True}

        # --- Handle final utterance (only process these to avoid duplicates) ---
        if event == "transcript.data":
            print(f"[Utterance] {speaker}: {text}")
            
            if not text.strip():
                return {"ok": True}

            lower = text.lower()
            
            # --- State: IDLE - listening for wake word ---
            if state == CaptureState.IDLE:
                if WAKE_WORD in lower:
                    # Debounce: avoid triggering too soon after last capture
                    if now - last_capture_end_ts < 2.0:
                        print(f"[Wake] Ignoring '{WAKE_WORD}' - too soon after last capture")
                        return {"ok": True}
                    
                    print(f"[Wake] Heard '{WAKE_WORD}' - starting capture")
                    state = CaptureState.CAPTURING
                    captured_utterances.clear()
                    last_utterance_ts = now
                    
                    # Check if question is in same utterance (after wake word)
                    parts = lower.split(WAKE_WORD, maxsplit=1)
                    if len(parts) > 1:
                        after_wake = parts[1].strip()
                        # Remove common punctuation
                        after_wake = after_wake.strip('.,!?;:')
                        if after_wake and len(after_wake) > 1:
                            captured_utterances.append(after_wake)
                            print(f"[Capture] Got initial text: {after_wake}")
                    
                return {"ok": True}
            
            # --- State: CAPTURING - accumulating utterances ---
            if state == CaptureState.CAPTURING:
                last_utterance_ts = now
                
                # Check for immediate finalization with "please" or similar end markers
                ends_with_please = re.search(r'\bplease[.?!,;:\s]*$', lower)
                ends_with_question = text.strip().endswith('?')
                
                # Don't re-capture the wake word if it appears again
                if WAKE_WORD not in lower:
                    # Check if "please" is in the middle of the utterance
                    if 'please' in lower and not ends_with_please:
                        # Split on "please" and take everything before it
                        before_please = re.split(r'\bplease\b', text, flags=re.I, maxsplit=1)[0].strip()
                        if before_please:
                            captured_utterances.append(before_please)
                            print(f"[Capture] Added (before 'please'): {before_please[:80]}")
                        
                        # Finalize immediately
                        asyncio.create_task(finalize_question_now())
                        return {"ok": True}
                    
                    captured_utterances.append(text.strip())
                    print(f"[Capture] Added: {text[:80]}")
                    
                    # Check for end markers
                    if ends_with_please or ends_with_question:
                        print(f"[Capture] Detected end marker, finalizing...")
                        asyncio.create_task(finalize_question_now())
                        return {"ok": True}
                
                # Check for silence timeout in background
                asyncio.create_task(check_capture_timeout())
                
                return {"ok": True}

        return {"ok": True}

    except Exception as e:
        print("[Webhook Error]", e)
        return {"error": str(e)}


async def finalize_question_now():
    """
    Immediately finalize the captured question (called when end marker detected).
    """
    global state, captured_utterances, last_capture_end_ts
    
    if state == CaptureState.CAPTURING and captured_utterances:
        question = " ".join(captured_utterances).strip()
        
        # Clean up "please" remnants and punctuation
        question = re.sub(r'\bplease[.?!,;:\s]*$', '', question, flags=re.I).strip()
        
        print(f"[Finalized Question] {question}")
        
        # Reset state
        state = CaptureState.IDLE
        last_capture_end_ts = time.time()
        captured_utterances.clear()
        
        # Process the question
        asyncio.create_task(handle_question_and_speak(question))


async def check_capture_timeout():
    """
    Background task to check if we should finalize the captured question
    after a period of silence.
    """
    global state, captured_utterances, last_utterance_ts, last_capture_end_ts
    
    await asyncio.sleep(SILENCE_TIMEOUT_SEC + 0.5)
    
    if state == CaptureState.CAPTURING:
        time_since_last = time.time() - last_utterance_ts
        
        if time_since_last >= SILENCE_TIMEOUT_SEC:
            # Finalize the question
            if captured_utterances:
                question = " ".join(captured_utterances).strip()
                
                # Clean up "please" remnants and punctuation
                question = re.sub(r'\bplease[.?!,;:\s]*$', '', question, flags=re.I).strip()
                
                print(f"[Finalized Question (timeout)] {question}")
                
                # Reset state
                state = CaptureState.IDLE
                last_capture_end_ts = time.time()
                captured_utterances.clear()
                
                # Process the question
                asyncio.create_task(handle_question_and_speak(question))
            else:
                print("[Capture] Timeout but no utterances captured")
                state = CaptureState.IDLE

# --- OpenAI client ---
client = OpenAI(api_key=OPENAI_API_KEY)

conversation_history = deque(maxlen=5)


# --- Heuristic classifier for MCP routing ---
def requires_mcp(question: str) -> bool:
    """
    Simple keyword-based heuristic for routing to MCP integrations
    (Google Calendar, GitHub, etc.).
    """
    q_lower = question.lower()

    # Calendar-related triggers
    calendar_keywords = [
        "calendar", "meeting", "schedule", "event", "remind", "invite", "zoom", "google meet"
    ]

    # GitHub-related triggers
    github_keywords = [
        "pull request", "commit", "repository", "repo", "branch", "merge",
        "issue", "push", "clone", "code review", "workflow", "action"
    ]

    if any(k in q_lower for k in calendar_keywords + github_keywords):
        return True

    # Also handle explicit “check”, “update”, or “open” requests that look like tool use
    if re.search(r"\b(check|update|create|delete|list|open)\b", q_lower):
        if "calendar" in q_lower or "github" in q_lower:
            return True

    return False


# --- MCP POST helper ---
def call_mcp_endpoint(prompt: str, bot_id: str) -> str:
    try:
        url = "http://100.124.227.16:5001/chat"
        payload = {
            "message": prompt,
            "bot_id": bot_id
        }
        headers = {"Content-Type": "application/json"}
        print("[MCP] Routing question to MCP endpoint...")
        resp = requests.post(url, headers=headers, json=payload, timeout=30)
        if resp.status_code == 200:
            data = resp.json()
            print("[MCP] Response:", json.dumps(data, indent=2))
            # Return stringified text if present
            if isinstance(data, dict):
                return data.get("response") or data.get("message") or str(data)
            return str(data)
        else:
            print("[MCP Error]", resp.status_code, resp.text[:200])
            return "Sorry, I couldn’t reach the MCP service."
    except Exception as e:
        print("[MCP Exception]", e)
        return "There was an error contacting the MCP."


async def call_openai_with_contextual_memory(question: str) -> str:
    try:
        msgs = [{"role": "system", "content": (
            "You are Nemo, a friendly, concise meeting assistant. "
            "Keep answers under 60 words, natural for speech."
        )}]
        for q, a in conversation_history:
            msgs.append({"role": "user", "content": q})
            msgs.append({"role": "assistant", "content": a})
        msgs.append({"role": "user", "content": question})
        print(f"[OpenAI] Searching: {question[:80]}...")
        resp = client.responses.create(model=OPENAI_MODEL, input=msgs)
        out = []
        for i in resp.output:
            if i.type == "message":
                for c in i.content:
                    if c.type == "output_text":
                        out.append(c.text)
        answer = " ".join(out).strip()
        return answer or "Sorry, I’m not sure."
    except Exception as e:
        print("[OpenAI Error]", e)
        return "Sorry, I hit an error."

# --- TTS ---
def elevenlabs_tts_to_mp3_bytes(text: str, voice_id="21m00Tcm4TlvDq8ikWAM") -> bytes:
    try:
        url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
        headers = {
            "Accept": "audio/mpeg",
            "xi-api-key": ELEVENLABS_API_KEY,
            "Content-Type": "application/json",
        }
        payload = {"text": text, "model_id": "eleven_multilingual_v2"}
        r = requests.post(url, headers=headers, json=payload, timeout=60)
        if r.status_code != 200:
            print("[TTS Error]", r.status_code, r.text[:200])
            return b""
        return r.content
    except Exception as e:
        print("[TTS Error]", e)
        return b""

# --- Recall output endpoints ---
def recall_output_audio(bot_id: str, mp3_bytes: bytes):
    print("[Recall] Sending TTS audio...")
    b64 = base64.b64encode(mp3_bytes).decode("utf-8")
    url = f"https://us-west-2.recall.ai/api/v1/bot/{bot_id}/output_audio/"
    headers = {"Authorization": RECALL_API_KEY, "content-type": "application/json"}
    payload = {"kind": "mp3", "b64_data": b64}
    requests.post(url, headers=headers, json=payload, timeout=30)


# --- Full pipeline ---
async def handle_question_and_speak(question: str):
    try:
        # Decide whether to route through MCP
        if requires_mcp(question):
            print("[Router] Using MCP for:", question)
            print("[A] On it.")
            # Synchronously call MCP API
            mcp_response = await asyncio.to_thread(call_mcp_endpoint, question, CURRENT_BOT_ID)
            print(f"[MCP Answer] {mcp_response}")
            # No audio output; just continue listening
            return
        else:
            print("[Router] Using LLM for:", question)
            answer = await call_openai_with_contextual_memory(question)
            conversation_history.append((question, answer))
            print(f"[A] {answer}")
            mp3 = await asyncio.to_thread(elevenlabs_tts_to_mp3_bytes, answer)
            if CURRENT_BOT_ID and mp3:
                recall_output_audio(CURRENT_BOT_ID, mp3)
    except Exception as e:
        print("[Pipeline Error]", e)


# --- ngrok + Recall bot setup ---
def start_ngrok(port: int = 8000) -> str:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    public_tunnel = ngrok.connect(addr=port, proto="http")
    print(f"[ngrok] Public URL: {public_tunnel.public_url}")
    return public_tunnel.public_url

def create_recall_bot(public_url: str) -> str:
    url = "https://us-west-2.recall.ai/api/v1/bot/"
    headers = {
        "Authorization": RECALL_API_KEY,
        "accept": "application/json",
        "content-type": "application/json",
    }

    SILENT_MP3_B64 = (
        "SUQzAwAAAAAAF1RTU0UAAAAPAAADTGF2ZjU2LjI0LjEwNAAAAAAAAAAAAAAA//tQxAADBQAA"
        "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
        "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
    )

    payload = {
        "meeting_url": MEETING_URL,
        "start_bot": True,
        "bot_name": "Nemo",
        "recording_config": {
            "transcript": {
                "provider": {
                    "deepgram_streaming": {
                        "model": "nova",
                        "language": "en",
                        "punctuate": "true",
                        "smart_format": "true",
                        "vad_turnoff_mode": "none",
                        "endpointing": 5000,
                    }
                },
                "diarization": {
                    "use_separate_streams_when_available": False
                },
            },
            "realtime_endpoints": [
                {
                    "type": "webhook",
                    "url": f"{public_url}/rt",
                    "events": [
                        "transcript.partial_data",
                        "transcript.data",
                    ],
                    "metadata": {},
                }
            ],
            "retention": {"type": "forever"},
        },
        "automatic_audio_output": {
            "in_call_recording": {
                "data": {"kind": "mp3", "b64_data": SILENT_MP3_B64}
            }
        },
    }

    r = requests.post(url, headers=headers, json=payload, timeout=60)
    if r.status_code >= 300:
        raise RuntimeError(f"Failed to create bot: {r.status_code} {r.text}")
    data = r.json()
    print(f"[Recall] Bot created successfully: {data['id']}")
    return data["id"]




# --- Run ---
nest_asyncio.apply()
CURRENT_BOT_ID = None

async def main():
    global CURRENT_BOT_ID
    port = 8000
    public_url = start_ngrok(port)
    CURRENT_BOT_ID = create_recall_bot(public_url)
    print(f"[Ready] Say '{WAKE_WORD}' followed by your question. The bot will capture after {SILENCE_TIMEOUT_SEC}s of silence.")
    config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

await main()


zsh:1: no matches found: pyngrok==7.*
[ngrok] Public URL: https://unministrative-twirly-ephraim.ngrok-free.dev
[Recall] Bot created successfully: d24fdc50-eb99-46aa-a55e-c6190f468101
[Ready] Say 'nemo' followed by your question. The bot will capture after 2.0s of silence.


INFO:     Started server process [68240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
t=2025-10-26T04:00:12-0700 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


[Partial] Ishan Buyyanapragada: Hello.
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Utterance] Ishan Buyyanapragada: Hello?
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Partial] Ishan Buyyanapragada: Hello. Hello.
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Partial] Ishan Buyyanapragada: Hello. Hello. Hello. Hello.
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Utterance] Ishan Buyyanapragada: Hello. Hello. Hello. Hello.
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Partial] Ishan Buyyanapragada: Hey, Nemo. Can
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Partial] Ishan Buyyanapragada: Hey, Nemo. Can you create for me a
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Partial] Ishan Buyyanapragada: Hey, NEEMO. Can you create for me a calendar event at five
INFO:     54.218.105.73:0 - "POST /rt HTTP/1.1" 200 OK
[Utterance] Ishan Buyyanapragada: Hey, NEEMO. Can you create for me a calendar event at
INFO:     54.218.10

t=2025-10-26T04:32:01-0700 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=59cf9f31b0a2 err="read EOF from remote peer"
t=2025-10-26T04:32:11-0700 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=884aafe1a82f clientid=fabe27f627f878a8f32b4de776b67e05
t=2025-10-26T04:47:39-0700 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=59cf9f31b0a2 err="read tcp 10.72.3.8:57641->3.136.132.147:443: read: connection reset by peer"
t=2025-10-26T04:47:54-0700 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=dd289c009420 clientid=fabe27f627f878a8f32b4de776b67e05
t=2025-10-26T04:53:02-0700 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=59cf9f31b0a2 err="read tcp 10.72.3.8:57758->3.12.62.205:443: read: connection reset by peer"
t=2025-10-26T04:53:17-0700 lvl=eror msg="heartbeat timeout, terminating session" obj

In [100]:
from pyngrok import ngrok
ngrok.kill()  # kills all active ngrok sessions

In [101]:
public_url = start_ngrok(8000)

[ngrok] Public URL: https://unministrative-twirly-ephraim.ngrok-free.dev
